### End to End demo of starting spot instance in AWS

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

#### Define parameters

In [3]:
name='fast-ai'

#### Get Existing VPC by tag name

In [4]:
vpc = get_vpc(name); vpc

#### Request Spot instance

In [5]:
instance_name = f'{name}-instance'

In [6]:
instance = create_spot_instance(name, vpc, spot_price='0.5', instance_type='t2.micro'); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-0438c1b19ba3e2b97
Rebooting...
Completed. SSH:  ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@34.215.171.83


ec2.Instance(id='i-0438c1b19ba3e2b97')

Request on demand instance (if spot error)

In [ ]:
# instance = create_instance(name, vpc, instance_type='t2.micro'); instance

In [7]:
instance = get_instance(f'{name}'); instance

ec2.Instance(id='i-0438c1b19ba3e2b97')

In [15]:
instance.wait_until_exists()

#### Attach EBS volume (Optional)

In [8]:
volume_tag = 'fast-ai-ebs-volume-test'

In [9]:
_ = attach_volume(instance, volume_tag, device='/dev/xvdf')

Volume attached. Please make sure to ssh into instance to format (if new volume) and mount


### SSH into instance

In [16]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


#### Mount EBS

In [17]:
upload_path = Path.cwd()/'upload_scripts/mount_ebs.sh'
upload_file(client, str(upload_path), 'mount_ebs.sh')

In [18]:
out, _ = run_command(client, 'chmod 755 mount_ebs.sh')

run_command returned: 



In [19]:
out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf') # no reformatting
# out, _ = run_command(client, './mount_ebs.sh --device /dev/xvdf --reformat true')

run_command returned: 



In [20]:
out, _ = run_command(client, 'ls ebs_mount_point') # no reformatting

run_command returned: 
saved.txt



#### Mount EFS

In [21]:
efs_addr = get_efs_address('fast-ai-efs'); efs_addr

'fs-0ea233a7.efs.us-west-2.amazonaws.com'

In [22]:
out, _ = run_command(client, 'mkdir ~/efs_mount_point')

run_command returned: 



In [23]:
efs_mount_cmd = f'sudo mount -t nfs -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2 {efs_addr}:/ ~/efs_mount_point'

In [ ]:
out, _ = run_command(client, efs_mount_cmd)

In [25]:
out, _ = run_command(client, 'ls efs_mount_point') # no reformatting

run_command returned: 
efs_saved.txt



### TODO

* tmux
* Auto shutdown after training
* Convert to script